In [1]:
import tensorflow as tf

In [2]:
print(tf.__version__)

1.15.0


In [0]:
from tensorflow.keras.datasets import cifar10
import sys
from tensorflow.keras import layers

In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 6s 0us/step


In [0]:
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

In [0]:
class TFRecords:   
#   def __init__(self, out_filename, images, labels):
#     self.out_filename = out_filename
#     self.images = images
#     self.labels = labels

  def _int64_feature(self, value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

  def _bytes_feature(self, value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
    
  def createDataRecord(self, out_filename, images, labels):

    writer = tf.io.TFRecordWriter(out_filename)

    for i in range(len(images)):
      feature = {
          'image_raw': self._bytes_feature(images[i].tostring()),
          'label': self._int64_feature(labels[i])
      }

      example = tf.train.Example(features=tf.train.Features(feature=feature))

      writer.write(example.SerializeToString())

    writer.close()
    sys.stdout.flush()

In [0]:
# Instantite the class

tfRecords = TFRecords()

In [0]:
tfRecords.createDataRecord('train.tfrecords', x_train, y_train)

In [0]:
tfRecords.createDataRecord('test.tfrecords', x_test, y_test)

In [10]:
!ls

sample_data  test.tfrecords  train.tfrecords


In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 5
num_filter = 22
compression = 0.5
dropout_rate = 0.2

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 22, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(5):
        BatchNorm = layers.BatchNormalization()(temp)
        relu = layers.Activation('relu')(BatchNorm)
        Conv2D_3_3 = layers.Conv2D(int(num_filter), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = layers.Dropout(dropout_rate)(Conv2D_3_3)
        concat = layers.Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 22, dropout_rate = 0.2):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = layers.Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = layers.Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = layers.AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    AvgPooling = layers.AveragePooling2D(pool_size=(2,2))(relu)
    flat = layers.Flatten()(AvgPooling)
    output = layers.Dense(num_classes, activation='softmax')(flat)
    
    return output

In [15]:
num_filter = 22
dropout_rate = 0.2
l = 5
input = layers.Input(shape=(img_height, img_width, channel,))
First_Conv2D = layers.Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [16]:
model = tf.keras.Model(inputs=[input], outputs=[output])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 22)   594         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 22)   88          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 22)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [0]:
model.compile(optimizer=tf.train.AdamOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [18]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 22)   594         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 22)   88          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 22)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [0]:
def parser(record):
  keys_to_features = {
      'image_raw': tf.FixedLenFeature((), tf.string),
      'label': tf.FixedLenFeature((), tf.int64)
  }
  parsed = tf.parse_single_example(record, keys_to_features)
  image = tf.decode_raw(parsed['image_raw'], tf.uint8)
  image = tf.cast(image, tf.int32)
  image = tf.reshape(image, [32,32,3])
  label = tf.cast(parsed["label"], tf.int32)
  label = tf.one_hot(label, 10)
  return image, label

In [0]:
def get_dataset_from_TFRecord(filename):
  dataset = tf.data.TFRecordDataset(filenames='train.tfrecords')
  dataset = dataset.map(parser)
  return dataset    

In [21]:
dataset = get_dataset_from_TFRecord('train.tfrecords')
dataset = dataset.batch(512)
dataset = dataset.repeat()

In [0]:
val_dataset = get_dataset_from_TFRecord('test.tfrecords')

In [0]:
val_dataset = val_dataset.batch(512).repeat()

In [24]:
model.fit(dataset, epochs=50, steps_per_epoch=30,
          validation_data=val_dataset,validation_steps=3)

Train on 30 steps, validate on 3 steps
Epoch 1/50
30/30 [==============================] - 13s 419ms/step - loss: 2.0585 - acc: 0.2375 - val_loss: 2.6884 - val_acc: 0.1224
Epoch 2/50
30/30 [==============================] - 5s 162ms/step - loss: 1.6597 - acc: 0.3774 - val_loss: 1.9987 - val_acc: 0.2624
Epoch 3/50
30/30 [==============================] - 5s 161ms/step - loss: 1.5518 - acc: 0.4254 - val_loss: 2.1144 - val_acc: 0.2044
Epoch 4/50
30/30 [==============================] - 6s 187ms/step - loss: 1.4131 - acc: 0.4772 - val_loss: 1.5834 - val_acc: 0.4225
Epoch 5/50
30/30 [==============================] - 5s 162ms/step - loss: 1.3342 - acc: 0.5061 - val_loss: 1.4345 - val_acc: 0.4824
Epoch 6/50
30/30 [==============================] - 5s 162ms/step - loss: 1.2884 - acc: 0.5313 - val_loss: 1.7363 - val_acc: 0.3659
Epoch 7/50
30/30 [==============================] - 5s 161ms/step - loss: 1.2191 - acc: 0.5534 - val_loss: 1.6773 - val_acc: 0.3789
Epoch 8/50
30/30 [==================

In [25]:
model.fit(dataset, epochs=50, steps_per_epoch=30,
          validation_data=val_dataset,validation_steps=3)

Train on 30 steps, validate on 3 steps
Epoch 1/50
30/30 [==============================] - 8s 270ms/step - loss: 0.5715 - acc: 0.7992 - val_loss: 0.7021 - val_acc: 0.7682
Epoch 2/50
30/30 [==============================] - 5s 161ms/step - loss: 0.5937 - acc: 0.7926 - val_loss: 0.7648 - val_acc: 0.7539
Epoch 3/50
30/30 [==============================] - 5s 162ms/step - loss: 0.6028 - acc: 0.7884 - val_loss: 0.9227 - val_acc: 0.7357
Epoch 4/50
30/30 [==============================] - 5s 160ms/step - loss: 0.5636 - acc: 0.8015 - val_loss: 0.5840 - val_acc: 0.8112
Epoch 5/50
30/30 [==============================] - 5s 162ms/step - loss: 0.5746 - acc: 0.7969 - val_loss: 0.7756 - val_acc: 0.7572
Epoch 6/50
30/30 [==============================] - 5s 161ms/step - loss: 0.5714 - acc: 0.8022 - val_loss: 0.7879 - val_acc: 0.7526
Epoch 7/50
30/30 [==============================] - 5s 161ms/step - loss: 0.5585 - acc: 0.8041 - val_loss: 0.5878 - val_acc: 0.7975
Epoch 8/50
30/30 [===================